In [6]:
def extraction(csv_entree, csv_sortie_ambig, csv_sortie_inc, stats_txt):
    
    """
        Fonction permettant, depuis un CSV extrait avec les colonnes
            - ID (numéro de token)
            - TOKEN (mot_forme)
            - LEMME (lemme ou liste des lemmes, séparés par des '|'
            - POS (pos ou liste des pos, séparés par des '|',
        
        1 - d'extraire les tokens où subsiste une ambiguïté sur le POS
        dans un nouveau CSV avec le contexte (3 tokens à gauche,
        3 tokens à droite, ainsi que leurs POS respectifs)
        
        2 - de calculer :
        - le total de tokens dans le fichier,
        - le pourcentage de lemmes résolus,
        - le pourcentage de pos résolus,
        - le nombre de lemmes inconnus,
        - le pourcentage de tokens extraits car toujours ambigus.
        
        :param csv_entree: Le chemin interne du fichier CSV dont on
            souhaite extraire les tokens et calculer les pourcentages.
        :param csv_sortie: Le chemin interne du fichier CSV où seront
            extraits les tokens encore ambigus.
    """
        
    import csv
    
    # Dictionnaires servant à l'écriture du nouveau CSV avec tokens ambigus.
    dico_mots = {}
    dico_local = {}
    dico_ambig = {}
    dico_inc = {}
    
    # Compteur pour les identifiants de dico_mots.
    identifiant = 1
    
    # Séparateur de valeurs pour les colonnes contenant le POS.
    intervale = " - "
    
    # Colonnes du futur fichier CSV avec tokens ambigus.
    colonnes = ['POS g', 'G1', 'G2', 'G3', 'TOKEN', 'D1', 'D2', 'D3', 'POS d']
    
    # Compteurs servant à l'établissement des pourcentages.
    nb_total_tokens = 0
    pos_ambigus = 0
    lemmes_uniques = 0
    pos_uniques = 0
    lemmes_inc = 0
    
    
    # Lecture du fichier CSV d'entrée et stockage de l'information dans csv_lu.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire, delimiter=";") #
        
        for row in csv_lu:
            
            # Établissement du dictionnaire servant à la création du nouveau CSV.
            dico_mots[int(row['ID'])] = row
            
            # Calcul du nombre de lemmes et pos résolus, ainsi que du total de tokens.
            if "|" not in row['LEMMES']:
                lemmes_uniques += 1
            if "|" not in row['POS']:
                pos_uniques += 1
            if row['LEMMES'] == 'INC':
                lemmes_inc += 1
            nb_total_tokens += 1
    
    
    # On boucle sur le dictionnaire des tokens pour créer le concordancier.
    for identifiant in dico_mots.keys():
        
        # Création des identifiants des contextes les plus éloignés.
        g1 = identifiant - 3
        d3 = identifiant + 3
        
        # Conditions : on ne créera ces entrées de dictionnaire que si le POS est ambigu,
        # et s'il y a bien trois tokens de contexte à inclure de chaque côté
        # (ce qui sera le cas à partir du token n°4 et jusqu'au token n°-4).
        
        if g1 in dico_mots.keys() and d3 in dico_mots.keys() and "|" in dico_mots[identifiant]['POS']:
            
            # Si les conditions sont réunies, on peut ajouter 1 au compteur de POS ambigus.
            pos_ambigus += 1
            
            # On crée les identifiants des autres tokens de contexte.
            g2 = identifiant - 2
            g3 = identifiant - 1
            d1 = identifiant + 1
            d2 = identifiant + 2
            
            # On crée une liste des POS pour le contexte gauche et droit, séparément.
            posg = [dico_mots[g1]['POS'], dico_mots[g2]['POS'], dico_mots[g3]['POS']]
            posd = [dico_mots[d1]['POS'], dico_mots[d2]['POS'], dico_mots[d3]['POS']]
            
            # On ajoute au dictionnaire local les valeurs construites
            # d'après le token et son contexte.
            
            dico_local['posG'] = intervale.join(posg)
            dico_local['G1'] = dico_mots[g1]['TOKEN']
            dico_local['G2'] = dico_mots[g2]['TOKEN']
            dico_local['G3'] = dico_mots[g3]['TOKEN']
            dico_local['token'] = dico_mots[identifiant]['TOKEN'] + " - " + str(identifiant)
            dico_local['D1'] = dico_mots[d1]['TOKEN']
            dico_local['D2'] = dico_mots[d2]['TOKEN']
            dico_local['D3'] = dico_mots[d3]['TOKEN']
            dico_local['posD'] = intervale.join(posd)
            
            # On ajoute le dictionnaire pour le token en cours
            # dans le dictionnaire final, comme valeur,
            # avec pour identifiant celui du token central.
            
            dico_ambig[identifiant] = dico_local
            identifiant += 1
            dico_local = {}
            
        elif g1 in dico_mots.keys() and d3 in dico_mots.keys() and dico_mots[identifiant]['POS'] == "Inconnu":
            
            # On crée les identifiants des autres tokens de contexte.
            g2 = identifiant - 2
            g3 = identifiant - 1
            d1 = identifiant + 1
            d2 = identifiant + 2
            
            # On crée une liste des POS pour le contexte gauche et droit, séparément.
            posg = [dico_mots[g1]['POS'], dico_mots[g2]['POS'], dico_mots[g3]['POS']]
            posd = [dico_mots[d1]['POS'], dico_mots[d2]['POS'], dico_mots[d3]['POS']]
            
            # On ajoute au dictionnaire local les valeurs construites
            # d'après le token et son contexte.
            
            dico_local['posG'] = intervale.join(posg)
            dico_local['G1'] = dico_mots[g1]['TOKEN']
            dico_local['G2'] = dico_mots[g2]['TOKEN']
            dico_local['G3'] = dico_mots[g3]['TOKEN']
            dico_local['token'] = dico_mots[identifiant]['TOKEN'] + " - " + str(identifiant)
            dico_local['D1'] = dico_mots[d1]['TOKEN']
            dico_local['D2'] = dico_mots[d2]['TOKEN']
            dico_local['D3'] = dico_mots[d3]['TOKEN']
            dico_local['posD'] = intervale.join(posd)
            
            # On ajoute le dictionnaire pour le token en cours
            # dans le dictionnaire final, comme valeur,
            # avec pour identifiant celui du token central.
            
            dico_inc[identifiant] = dico_local
            identifiant += 1
            dico_local = {}
        
    # On ouvre le CSV de sortie en mode écriture et on écrit les noms des colonnes.
    
    with open(csv_sortie_ambig, 'w') as csv_a_ecrire:
        a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames = colonnes)
        # Écrire les noms des colonnes.
        a_ecrire.writeheader()
        
        # On boucle sur les clés du dictionnaire final.
        for identifiant in dico_ambig.keys():
            
            a_ecrire.writerow({
                'POS g' : dico_ambig[identifiant]['posG'],
                'G1' : dico_ambig[identifiant]['G1'],
                'G2' : dico_ambig[identifiant]['G2'],
                'G3' : dico_ambig[identifiant]['G3'],
                'TOKEN' : dico_ambig[identifiant]['token'],
                'D1' : dico_ambig[identifiant]['D1'],
                'D2' : dico_ambig[identifiant]['D2'],
                'D3' : dico_ambig[identifiant]['D3'],
                'POS d' : dico_ambig[identifiant]['posD']
            })
    
    with open(csv_sortie_inc, 'w') as csv_a_ecrire:
        a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames = colonnes)
        # Écrire les noms des colonnes.
        a_ecrire.writeheader()
        
        # On boucle sur les clés du dictionnaire final.
        for identifiant in dico_inc.keys():
            
            a_ecrire.writerow({
                'POS g' : dico_inc[identifiant]['posG'],
                'G1' : dico_inc[identifiant]['G1'],
                'G2' : dico_inc[identifiant]['G2'],
                'G3' : dico_inc[identifiant]['G3'],
                'TOKEN' : dico_inc[identifiant]['token'],
                'D1' : dico_inc[identifiant]['D1'],
                'D2' : dico_inc[identifiant]['D2'],
                'D3' : dico_inc[identifiant]['D3'],
                'POS d' : dico_inc[identifiant]['posD']
            })
    
    # L'écriture du CSV étant terminée, on calcule les pourcentages.
    
    pourcentage_lemmes = lemmes_uniques * 100 / nb_total_tokens
    pourcentage_pos = pos_uniques * 100 / nb_total_tokens
    pourcentage_lemmes_inc = lemmes_inc * 100 / nb_total_tokens
    pourcentage_extraits = pos_ambigus * 100 / (nb_total_tokens-4)
    
    # On donne le résultat des calculs de pourcentages.
    
    with open(stats_txt, 'w') as txttobe:
        
        txttobe.write("À partir du fichier '" + csv_entree.split("/")[-1] + "' :\n\n")
        txttobe.write(str(round(pourcentage_lemmes,2)) + "% de lemmes uniques.\n")
        txttobe.write(str(round(pourcentage_pos,2)) + "% de POS uniques.\n")
        txttobe.write(str(lemmes_inc) + " lemmes inconnus, soit " + str(round(pourcentage_lemmes_inc,2)) + "%.\n")
        txttobe.write(str(pos_ambigus) + "(" + str(round(pourcentage_extraits,2)) + "%) POS ambigus extraits dans '" + csv_sortie_ambig.split("/")[-1] + "'.\n")

In [7]:
extraction('/home/erminea/Documents/CONDE/Encodage/basnage_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/basnage_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/basnage_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/basnage_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/berault_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/berault_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/berault_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/berault_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/instructions_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/instructions_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/instructions_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/instructions_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/merville_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/merville_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/merville_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/merville_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/pesnelle_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/pesnelle_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/pesnelle_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/pesnelle_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/rouille_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/rouille_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/rouille_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/rouille_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/ruines_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/ruines_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/ruines_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/ruines_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/tac_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/tac_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/tac_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/tac_stats.txt')

extraction('/home/erminea/Documents/CONDE/Encodage/terrien_tokens.csv',
          '/home/erminea/Documents/CONDE/Encodage/terrien_tokens_ambig.csv',
          '/home/erminea/Documents/CONDE/Encodage/terrien_tokens_inc.csv',
          '/home/erminea/Documents/CONDE/Encodage/terrien_stats.txt')